In [1]:
from datasets import load_dataset
ds_train = load_dataset("trl-lib/ultrafeedback_binarized", split="train")

/home/nam/miniconda3/envs/diff/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [40]:
ds_train[0]

{'chosen': [{'content': 'Use the pygame library to write a version of the classic game Snake, with a unique twist',
   'role': 'user'},
  {'content': "Sure, I'd be happy to help you write a version of the classic game Snake using the pygame library! Here's a basic outline of how we can approach this:\n\n1. First, we'll need to set up the game display and create a game object that we can use to handle the game's state.\n2. Next, we'll create the game's grid, which will be used to represent the game board. We'll need to define the size of the grid and the spaces within it.\n3. After that, we'll create the snake object, which will be used to represent the player's movement. We'll need to define the size of the snake and the speed at which it moves.\n4. We'll also need to create a food object, which will be used to represent the food that the player must collect to score points. We'll need to define the location of the food and the speed at which it moves.\n5. Once we have these objects se

In [1]:
import pandas as pd

df = pd.read_csv("hf://datasets/Programmer-RD-AI/genz-slang-pairs-1k/genz_dataset.csv")
df.head()

/home/nam/miniconda3/envs/diff/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


,normal,gen_z
0,"I'm really tired today, I think I need some rest.","I'm totally drained today, need to catch some ..."
1,"I'm really tired today, I just want to relax a...","I'm hella drained today, just wanna chill at h..."
2,I'm really excited for the concert tonight.,I'm so hype for the concert tonight.
3,"I'm really tired today, I think I need some co...","I'm so drained today, I gotta get me some caff..."
4,I'm really looking forward to the weekend.,"I'm so hyped for the weekend, can't wait to tu..."


In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

model_name = "openai/gpt-oss-20b"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="cuda",
    torch_dtype=torch.bfloat16,
)
#  This takes up around 13GB VRAM
tokenizer.add_special_tokens({'additional_special_tokens': ['<CUSTOM>']})
model.resize_token_embeddings(len(tokenizer))

/home/nam/miniconda3/envs/diff/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
`torch_dtype` is deprecated! Use `dtype` instead!
Loading checkpoint shards: 100%|██████████| 3/3 [00:02<00:00,  1.03it/s]


Embedding(200020, 2880, padding_idx=199999)

In [2]:
import re

def generate_gpt_prompt_from_template(normal_sentence):
    gpt_prompt_template_for_genz = f"<|start|>system<|message|>You are ChatGPT, a large language model trained by OpenAI. You must answer in 1 sentence and only what you are asked. DO NOT overanalyse this query. Assume whatever you want. You MUST answer with something ALWAYS.<|end|><|start|>user<|message|>How do you say '{normal_sentence}' in <CUSTOM> slang?<|end|>"
    return {'content': gpt_prompt_template_for_genz, 'role': 'user'}

def post_process_normal_gpt_response(gpt_response, genz_version):
    start_pattern_pure_output = "<|start|>assistant<|channel|>final<|message|>"
    end_pattern_pure_output = "<|return|>"
    pure_output = None
    if len(gpt_response.split("<|end|>")) != 2:
        print("This sample funny")
        print(gpt_response)
    analysis_tokens, output_tokens = gpt_response.split("<|end|>")
    pattern = re.escape(start_pattern_pure_output) + r"(.*?)" + re.escape(end_pattern_pure_output)
    match = re.search(pattern, output_tokens)
    if match:
        pure_output = match.group(1)
        # print(f"pure_output = {pure_output}")
        # print("match found")
        output_tokens = output_tokens.replace(pure_output, genz_version)
        if pure_output in analysis_tokens:
            # print("pure output is in analysis tokens")
            analysis_tokens = analysis_tokens.replace(pure_output, genz_version)

    return {'content': analysis_tokens + "<|end|>" + output_tokens, 'role': 'assistant'}

In [4]:
from pprint import pprint
import random
from tqdm import tqdm

custom_training_data = []
for i in tqdm(range(len(df))):
    skip_counter, skip_indexes = 0, 
    row = df.iloc[i]
    normal, genz = row["normal"], row["gen_z"]
    ip_for_gpt = generate_gpt_prompt_from_template(normal)
    inputs = tokenizer(ip_for_gpt['content'], return_tensors="pt", padding=True, truncation=True).to(model.device)
    generated = model.generate(**inputs, max_new_tokens=700)
    # pprint(tokenizer.decode(generated[0][inputs["input_ids"].shape[-1] :]))
    op_from_default_gpt = {'content': tokenizer.decode(generated[0][inputs["input_ids"].shape[-1] :]), 'role': 'assistant'}
    try:
        post_process_op = post_process_normal_gpt_response(op_from_default_gpt['content'], genz)
        # pprint(post_process_op)
        row_to_append = {'chosen': [ip_for_gpt, post_process_op],
                        'rejected': [ip_for_gpt, op_from_default_gpt],
                        'score_chosen': round(random.randint(7, 9)+ random.random(), 2),
                        'score_rejected': round(random.randint(1, 4) + random.random(), 2)
                        }
        # pprint(row_to_append)
        custom_training_data.append(row_to_append)
    except:
        pass

custom_data_hf_format_df = pd.DataFrame(custom_training_data)

  0%|          | 3/1005 [00:53<4:55:52, 17.72s/it]

This sample funny
<|start|>...<|end|><|start|>assistant<|channel|>analysis<|message|>The user says: "How do you say 'I'm really excited for the concert tonight.' in <CUSTOM> slang?" There's some <CUSTOM> tag. It's presumably something like a placeholder: a custom slang might maybe be "OMG I'm so hyped!!!" But it's ambiguous. The user may want the phrase in slanguage for an unknown slang. There's no defined slang provided. The user says "in <CUSTOM> slang". Possibly they want something like "I'm super psyched for the show tonight." But the tag <CUSTOM> perhaps indicates they want me to make up an inventive slang. But the instruction says: "You are ChatGPT... You must answer in 1 sentence and only what you are asked. You MUST answer ... Assume whatever you want." So I can assume "CUSTOM" is some slang. Let's produce a creative slang: "Yo, I'm lit AF for the concert tonight!" This is 1 sentence, that covers. There's no requirement to explain or anything else.<|end|><|start|>assistant<|cha

  0%|          | 4/1005 [01:37<6:46:48, 24.38s/it]


KeyboardInterrupt: 

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

import pandas as pd
from pprint import pprint
import random
from tqdm import tqdm
import re

from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

df = pd.read_csv("hf://datasets/Programmer-RD-AI/genz-slang-pairs-1k/genz_dataset.csv")
model_name = "openai/gpt-oss-20b"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="cuda",
    torch_dtype=torch.bfloat16,
)
tokenizer.add_special_tokens({'additional_special_tokens': ['<CUSTOM>']})
model.resize_token_embeddings(len(tokenizer))
#  This takes up around 13GB VRAM

def generate_gpt_prompt_from_template(normal_sentence):
    gpt_prompt_template_for_genz = f"<|start|>system<|message|>You are ChatGPT, a large language model trained by OpenAI. You must answer in 1 sentence and only what you are asked. DO NOT overanalyse the query. Assume whatever you want. You MUST answer with something ALWAYS. Reasoning: low<|end|><|start|>user<|message|>How do you say '{normal_sentence}' in <CUSTOM> slang?<|end|>"
    return {'content': gpt_prompt_template_for_genz, 'role': 'user'}

def post_process_normal_gpt_response(gpt_response, genz_version):
    start_pattern_pure_output = "<|start|>assistant<|channel|>final<|message|>"
    end_pattern_pure_output = "<|return|>"
    pure_output = None
    if len(gpt_response.split("<|end|>")) != 2:
        if "<|start|>???<|end|>" in gpt_response:
            print("Funny Sample -> That weird <|start|>???<|end|> pattern")
        elif len(gpt_response.split("<|end|>")) == 1:
            print("Did not finish thinking")
        else:
            print(f"This sample funny, splits into list of length {len(gpt_response.split('<|end|>'))}")
            print(gpt_response)
    analysis_tokens, output_tokens = gpt_response.split("<|end|>")
    pattern = re.escape(start_pattern_pure_output) + r"(.*?)" + re.escape(end_pattern_pure_output)
    match = re.search(pattern, output_tokens)
    if match:
        pure_output = match.group(1)
        # print(f"pure_output = {pure_output}")
        # print("match found")
        output_tokens = output_tokens.replace(pure_output, genz_version)
        if pure_output in analysis_tokens:
            # print("pure output is in analysis tokens")
            analysis_tokens = analysis_tokens.replace(pure_output, genz_version)

    return {'content': analysis_tokens + "<|end|>" + output_tokens, 'role': 'assistant'}



custom_training_data = []
for i in tqdm(range(10)):
    row = df.iloc[i]
    skip_counter, skip_indexes = 0, []
    normal, genz = row["normal"], row["gen_z"]
    ip_for_gpt = generate_gpt_prompt_from_template(normal)
    inputs = tokenizer(ip_for_gpt['content'], return_tensors="pt", padding=True, truncation=True).to(model.device)
    generated = model.generate(**inputs, max_new_tokens=700)
    # pprint(tokenizer.decode(generated[0][inputs["input_ids"].shape[-1] :]))
    op_from_default_gpt = {'content': tokenizer.decode(generated[0][inputs["input_ids"].shape[-1] :]), 'role': 'assistant'}
    try:
        post_process_op = post_process_normal_gpt_response(op_from_default_gpt['content'], genz)
        # pprint(post_process_op)
        row_to_append = {'chosen': [ip_for_gpt, post_process_op],
                        'rejected': [ip_for_gpt, op_from_default_gpt],
                        'score_chosen': round(random.randint(7, 9)+ random.random(), 2),
                        'score_rejected': round(random.randint(1, 2) + random.random(), 2)
                        }
        # pprint(row_to_append)
        custom_training_data.append(row_to_append)
    except:
        skip_counter += 1
        skip_indexes.append(i)
        pass

custom_data_hf_format_df = pd.DataFrame(custom_training_data)

custom_data_hf_format_df.to_csv("../datasets/mini_custom_genz_dataset_in_hf_format.csv")

/home/nam/miniconda3/envs/diff/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
`torch_dtype` is deprecated! Use `dtype` instead!
100%|██████████| 10/10 [01:03<00:00,  6.35s/it]


In [5]:
custom_data_hf_format_df = pd.DataFrame(custom_training_data)

custom_data_hf_format_df.to_csv("datasets/custom_genz_dataset_in_hf_format.csv")

In [ ]:
# messages = [
#     {"role": "system", "content": "You are a helpful agent."},
#     {"role": "user", "content": "What is the weather like in Madrid?"},
# ]
# inputs = tokenizer.apply_chat_template(
#     messages,
#     # add_generation_prompt=True,
#     return_tensors="pt",
#     return_dict=True,
# ).to(model.device)
# print(tokenizer.decode(generated[0][inputs["input_ids"].shape[-1] :]).split("<|end|><|start|>assistant<|channel|>final<|message|>")[-1]

# prompt = "<|start|>system<|message|>You are ChatGPT, a large language model trained by OpenAI." \
# "\nKnowledge cutoff: 2024-06\nCurrent date: 2026-02-27\n\nReasoning: medium\n\n# " \
# "Valid channels: analysis, commentary, final. Channel must be included for every message." \
# "<|end|><|start|>user<|message|> Say 'I'm really tired today, I think I need some rest.' in GenZ slang.<|end|>"

# prompt = "<|end|><|start|>user<|message|> Say 'I'm really tired today, I think I need some rest.' in GenZ slang.<|end|>"
prompt = "<|start|>system<|message|>You are ChatGPT, a large language model trained by OpenAI. Answer in 1 sentence and only what you are asked.<|end|><|start|>user<|message|>How do you say 'I'm really tired today, I think I need some rest.' in GenZ slang?<|end|>"

inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True).to(model.device)

generated = model.generate(**inputs, max_new_tokens=700)
print(tokenizer.decode(generated[0][inputs["input_ids"].shape[-1] :]))

<|start|>assistant<|channel|>analysis<|message|>We need to transform a sentence into GenZ slang. The user asks: "How do you say 'I'm really tired today, I think I need some rest.' in GenZ slang?" They want an answer. Provide a GenZ slang version. "I'm feeling wiped out today, need some chill vibes." Something like that. Perhaps "I'm straight up beat today, need some chill." Should be one sentence. The assistant should answer: "Low key fried today, gotta catch some z's." Or "I'm shook, totally wiped, need some rest." Must be in GenZ slang. Use words like "woke" and "lit"? The best answer: "Low-key fried today, need to get some Z's." Let's produce that.<|end|><|start|>assistant<|channel|>final<|message|>Low‑key fried today—gonna catch some Zs and recharge.<|return|>


In [ ]:
from datasets import Dataset

data = {
    "pokemon": ["bulbasaur", "squirtle", "charmander"],
    "type": ["grass", "water", "fire"],
    "hp": [45, 44, 39]
}

hf_dataset = Dataset.from_dict(data)

print(hf_dataset)
print(hf_dataset[0])


In [ ]:
from trl import DPOTrainer
from datasets import load_dataset

trainer = DPOTrainer(
    model="openai/gpt-oss-20b",
    train_dataset=load_dataset("trl-lib/ultrafeedback_binarized", split="train"),
)
trainer.train()